# Standalone Correction Function Implementation

### Quickly obtrain a corrected value for a predicted [B] given the predicted [C]

In [1]:
import numpy as np
import pandas as pd
import os
import scipy

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

import seaborn as sns
sns.set(style='whitegrid')

from os.path import join as pjoin

import itertools as it

%config InteractiveShell.ast_node_interactivity='all'
%config InlineBackend.figure_format = 'svg'

In [2]:
# Correct and Model

# Corrected

# Import Correction Data
# Create Correction Models
# Both Data has to be Corrected

# path of the correction factor parameters
path = '../data_o/calibration/community_calibration/created_models/reqd_corr_funcs_parameters.csv'

def get_corr_params(file_loc, model='A680_A750_560'):
    
    print(model)
    
    df = pd.read_csv(path, index_col=0)
    
    return df.loc[model].values

def  corr_func_Bsc(pB, pC, parameters, which_a2=None):
    
    # There are 3 options about which a2 to use,
    # a2 refers to the breadth of the correcting
    # parabola. These are either the:
        # mean
        # median
        # a2 @ [B]= 0 (Row12)
    
    
    a2s = parameters[:-2]
    hfit = parameters[-2:]
    # WHICH Parabolic BREADTH to use for correction
    if which_a2 =='mean':
        a2 = np.mean(a2s) # this is the mean
        
    elif which_a2 =='median':
        a2 = np.median(a2s) # this should be more robust to outliers
    
    # set as default
    else:
        a2 = -0.499525586 #global over 3 models 

    # given the fitted line avove, use poly1d to create a 
    # prediction function
    htor = np.poly1d(hfit)
    
    # takes in the predicted pB and spits out h in (h, k) the parabola
    # that lives at this pB... i.e. vertex is at (h,pB)
    lH = htor(pB)
    
    
    # IMPORTANT #
    # this is from an algebraic manipulation of the parabolic function
    # in polynomial and parabolic form where h = -b/2*a2 --- since we
    # "know: a2 and p_h is dependant on the concetration of B we 
    # estimate the value of b in f(x) = a2*x^2 + b*x + c
    b = np.multiply(-2*lH,a2 )    
    cv_tor = np.poly1d([a2, b, 0])
    
    return cv_tor(pC)

# Input Values or Toy Examples

In [11]:
# predicted values
pB = 0.1443
pC = 0.5124

In [12]:
parameters = get_corr_params(path, 'A680_A720_560')

A680_A720_560


In [13]:
corr_func_Bsc(pB, pC, parameters, None)

0.02595096357340132

In [14]:
# Corrected pB

pB + corr_func_Bsc(pB, pC, parameters, None)

0.17025096357340133